# 02 – Cell Segmentation

This notebook shows how to perform cell segmentation using the `CellSegmenter`.  For demonstration, the fallback classical method is used.  Adjust `image_path` to your image file.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from isletscope.segmentation import CellSegmenter

image_path = 'path/to/your/image.tif'
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f'Cannot load image at {image_path}')

segmenter = CellSegmenter()
result = segmenter.segment(img)
mask = result['mask']
labels = result['labels']

# Display results
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')
axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Binary mask')
axes[1].axis('off')
axes[2].imshow(labels, cmap='nipy_spectral')
axes[2].set_title('Labeled components')
axes[2].axis('off')
plt.show()